<a href="https://colab.research.google.com/github/kwanhong66/TodayILearned/blob/master/deep_learning_basic_with_pytorch/mnist_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
torch.cuda.is_available()

True

In [0]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [5]:
# MNIST dataset
mnist_train = datasets.MNIST(root='MNIST_data/',
                             train=True,
                             transform=transforms.ToTensor(),
                             download=True)

mnist_test = datasets.MNIST(root='MNIST_data/',
                            train=False,
                            transform=transforms.ToTensor(),
                            download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

0it [00:00, ?it/s]

9920512it [00:01, 8518607.96it/s]                            


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 129943.06it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw


1654784it [00:00, 2146559.80it/s]                            
0it [00:00, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


8192it [00:00, 48873.09it/s]            


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


In [0]:
# Model
class CNN(nn.Module):
    def __init__(self) -> None:
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)

        out = out.view(out.size(0), -1)  # batch_size x flattened input
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [19]:
model = CNN().to(device)

value = torch.Tensor(1, 1, 28, 28).to(device)
print(model(value).shape)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

torch.Size([1, 10])


In [20]:
# training
total_batch = len(data_loader)
print('Learning Started.')

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {}] cost = {}'.format(epoch+1, avg_cost))

print('Learning Finished.')

Learning Started.
[Epoch: 1] cost = 0.14766906201839447
[Epoch: 2] cost = 0.0420551523566246
[Epoch: 3] cost = 0.027589818462729454
[Epoch: 4] cost = 0.021405737847089767
[Epoch: 5] cost = 0.01884468086063862
[Epoch: 6] cost = 0.013690690509974957
[Epoch: 7] cost = 0.01228722557425499
[Epoch: 8] cost = 0.010131857357919216
[Epoch: 9] cost = 0.00916285626590252
[Epoch: 10] cost = 0.0074213966727256775
[Epoch: 11] cost = 0.008496756665408611
[Epoch: 12] cost = 0.006626784335821867
[Epoch: 13] cost = 0.006562419701367617
[Epoch: 14] cost = 0.004642019979655743
[Epoch: 15] cost = 0.00542724272236228
Learning Finished.


In [21]:
# Test model and check accuracy
with torch.no_grad():
  model.eval() # eval mode

  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, dim=1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy: {}'.format(accuracy.item()))

Accuracy: 0.9749999642372131


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
